In [18]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from gensim.models.ldamodel import LdaModel
from gensim import corpora, models, similarities, matutils
from textblob import TextBlob

In [2]:
train_df = pd.read_csv("../data/train.csv")
train_df.head()

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67


In [3]:
attributes_df = pd.read_csv("../data/attributes.csv")
attributes_df.head()

,product_uid,name,value
0,100001,Bullet01,Versatile connector for various 90° connection...
1,100001,Bullet02,Stronger than angled nailing or screw fastenin...
2,100001,Bullet03,Help ensure joints are consistently straight a...
3,100001,Bullet04,Dimensions: 3 in. x 3 in. x 1-1/2 in.
4,100001,Bullet05,Made from 12-Gauge steel


In [4]:
product_descrition_df = pd.read_csv("../data/product_descriptions.csv")
product_descrition_df.head()

,product_uid,product_description
0,100001,"Not only do angles make joints stronger, they ..."
1,100002,BEHR Premium Textured DECKOVER is an innovativ...
2,100003,Classic architecture meets contemporary design...
3,100004,The Grape Solar 265-Watt Polycrystalline PV So...
4,100005,Update your bathroom with the Delta Vero Singl...


In [5]:
descriptions = product_descrition_df.product_description

In [6]:
stopset = stopwords.words('english')

In [7]:
texts = [[word for word in document.lower().replace(".", "").split() if word not in stopset ] for document in descriptions]

In [10]:
noun_text = []
for t in texts:
    NN_tags = TextBlob(" ".join(t)).noun_phrases
    noun_text.append(NN_tags)

In [11]:
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
            frequency[token] += 1
        
texts2 = [[token for token in text if frequency[token] > 1] for text in texts]

In [13]:
dictionary = corpora.Dictionary(texts2)
dictionary.save('descriptions.dict')

In [14]:
corpus = [dictionary.doc2bow(text) for text in texts2]
corpora.MmCorpus.serialize('descriptions.mm', corpus)

In [15]:
dictionary = corpora.Dictionary.load('descriptions.dict')
corpus = corpora.MmCorpus('descriptions.mm')

In [16]:
lda = LdaModel(corpus, id2word=dictionary, num_topics=100, passes=20)

In [17]:
lda.print_topics(num_topics = 100, num_words=10)

[(0,
  u'0.068*window + 0.034*screen + 0.031*windows + 0.016*strip + 0.016*strips + 0.012*vinyl + 0.011*easy + 0.011*storm + 0.010*insect + 0.010*white'),
 (1,
  u'0.035*flush + 0.018*laminate + 0.017*wooden + 0.015*proudly + 0.014*molding + 0.013*round + 0.012*manufactured + 0.012*transition + 0.011*staining + 0.011*moldings'),
 (2,
  u'0.038*sink + 0.035*kitchen + 0.022*ceramic + 0.016*brass + 0.016*faucet + 0.014*drain + 0.012*lifetime + 0.010*disc + 0.010*finish + 0.010*cartridge'),
 (3,
  u'0.018*oz + 0.016*lawn + 0.016*use + 0.016*grass + 0.011*visible + 0.010*leg + 0.010*pond + 0.010*unfinished + 0.010*formula + 0.010*used'),
 (4,
  u'0.038*water + 0.026*tub + 0.022*valve + 0.019*bath + 0.019*shower + 0.017*faucet + 0.013*faucets + 0.010*handle + 0.010*design + 0.009*kohler'),
 (5,
  u'0.043*mounting + 0.033*kit + 0.028*hardware + 0.023*wall + 0.020*installation + 0.018*rod + 0.018*includes + 0.017*mount + 0.015*included + 0.014*easy'),
 (6,
  u'0.073*air + 0.038*filter + 0.020*

In [24]:
df = pd.merge(train_df,product_descrition_df, on="product_uid")